# Create Infrastructure for your Machine Learning Project
With this code, we will create a Datastore, a Compute Cluster and an AKS Cluster for our Machine Learning Project.<p>
    <b>Datastores</b> are pointers that connect your Azure Machine Learning Service Workspace to your Azure Storage Accounts or Azure SQL Servers.<br><b>Compute Clusters</b> are clusters of virtual machines that you use to remotely train your Machine Learning models. <br><b>AKS Clusters</b> are clusters of virtual machines that you use to score your Models in real-time after you train them.<p>AKS clusters are necessary only if you intend to score data in real-time instead of batch and should only be used in production.

### Load your Libraries

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore, Keyvault
from azureml.core.compute import AksCompute, ComputeTarget, AmlCompute

### Check your version of the AzureML Python SDK

In [ ]:
print("You are currently using version " + azureml.core.VERSION + " of the Azure ML SDK")

### Set your Workspace
When performing this step for the first time, you will be asked to sign in using a hyperlink and a code generated below.<br>You will be asked to log in every so often depending on your organization.<br>  This connects your notebook to your Azure Machine Learning Service Workspace.

In [ ]:
# Retrieve your workspace from config
ws = Workspace.from_config()

### Set your Variables here.
This is the only place in the notebook where you have to adjust your code. There are 4 sections.<p>Section 1 is for creating a Datastore for an Azure Blob Container. This can be used for Blob Containers on Azure Data Lake as well.</p><p>Section 2 is for creating a Datastore for an Azure SQL Database.</p><p>Section 3 is for creating a Compute Cluster for training machine learning models remotely and for running Machine Learning Pipelines.</p><p>Section 4 is for creating an AKS Cluster to host Real-Time Scoring Endpoints.</p>

In [ ]:
# Section 1 - Blob Container Datastore

# Enter a name for your Datastore. It can include lowercase letters, numbers and underscores.
Blob_Datastore_Name = 'your_ds_blob_name'

# Enter the name of the Storage Account for which you will create an AMLS Datastore.
Storage_Account_Name = 'mlteachingstorage'

# Enter the name of the Container for which you will create an AMLS Datastore.
Storage_Container_Name = 'ml-teaching-datalake'

# Enter the Keyvault Secret Name. This Secret will hold your Storage Account Key.
Storage_Account_Key_Secret = 'datastore-password'

# Enter the Key for your Storage Account. You can delete this after you run this cell once.
Storage_Account_Key = 'your-storage-account-key'

# Set your Key Vault Secret which will hold your Storage Account Key. Comment this out after you run this cell once.
kv.set_secret(name = Storage_Account_Key_Secret, value = Storage_Account_Key)

In [ ]:
# Section 2 - Azure SQL Database Datastore

# Enter a name for your Datastore. It can include lowercase letters, numbers and underscores.
SQL_Datastore_Name = 'your_ds_sql_name'

# Enter the name of the SQL Server for which you will create an AMLS Datastore.
SQL_Server_Name = 'mlteachingstorage'

# Enter the name of the Azure SQL Database for which you will create an AMLS Datastore.
SQL_Database_Name = 'ml-teaching-datalake'

# Enter the Keyvault Secret Name. This Secret will hold your SQL User Name.
SQL_User_Secret = 'sql-username'

# Enter the Keyvault Secret Name. This Secret will hold your SQL User Password.
SQL_Password_Secret = 'sql-password'

# Enter your SQL Username. You can delete this after you run this cell once.
SQL_User_Name = 'your-sql-username'

# Enter your SQL Password. You can delete this after you run this cell once.
SQL_Password = 'your-sql-password'

# Set your Key Vault Secret which will hold your SQL Username. Comment this out after you run this cell once.
kv.set_secret(name = SQL_User_Secret, value = SQL_User_Name)

# Set your Key Vault Secret which will hold your SQL Password. Comment this out after you run this cell once.
kv.set_secret(name = SQL_Password_Secret, value = SQL_Password)

In [ ]:
# Section 3 - Compute Cluster

# Enter a name for your Compute Cluster. It can include letters, digits and dashes and be between 2-16 characters long.
Compute_Cluster_Name = 'your-CC-Name'

# Enter a Virtual Machine Type/Size/SKU. See a list of VMs by using the GUI and creating a Compute Instance or Cluster.
# Standard_DS3 VMs are CPU based. For Deep Learning tasks, use a GPU-based VM such as a Standard_NC6.
Compute_Cluster_Type = 'Standard_DS3_v2'

# Enter the minimum number of Virtual Machine Nodes for your Cluster. 0 is the recommended for cost savings.
Compute_Cluster_Min_Node_Count = 0

# Enter the maximum number of Nodes for your Cluster. 
# Set this higher for more parallelization in AutoML, HyperDrive, Dask runs, Many Models, and Parallel Run Pipelines.
Compute_Cluster_Max_Node_Count = 4

# Enter the number of seconds a Compute Cluster is not in use before it scales down.
# We recommend 1200 seconds (20 minutes) for development work and 120 seconds (2 minutes) for projects in production. 
Idle_Seconds = 1200

In [ ]:
# Section 4 - AKS Cluster

# Enter a name for your AKS Cluster. It can include letters, digits and dashes and be between 2-16 characters long.

AKS_Cluster_Name = 'your-AKS-Name'

# Enter a Virtual Machine Type/Size/SKU
AKS_Cluster_Type = 'Standard_DS3_v2'

# Enter the number of Virtual Machine Nodes for your Cluster. Core Count per Node x # of Nodes must be greater than 12.
AKS_Node_Count = 3

# Enter the Desired Azure Location of your AKS Cluster.
AKS_Location = 'centralus'

### Connect to Keyvault
Each Workspace comes with its own default Azure Keyvault.  Use this to store passwords, known as secrets.<br>Datastores require account keys for registration in the following step.

In [ ]:
kv = Keyvault(ws)

### Register a Datastore to link your Storage Account to your AMLS Workspace
A Datastore is a pointer to an Azure Storage Account or Azure SQL Database that links to your AMLS Workspace.<br>
In the case of a Storage Account, you connect directly to a blob container.  You will need the following:<br>
- Storage Account Name
- Storage Account Container Name
- Storage Account Key

In [ ]:
# Retrieve your Storage account Key Secret from Keyvault
storage_secret = kv.get_secret(Storage_Account_Key_Secret)

In [ ]:
# Set up a Datastore using the normal parameters (for Blob Storage or Azure Data Lake Gen 2)
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name= Blob_Datastore_Name,  
                                                         container_name= Storage_Container_Name, 
                                                         account_name= Storage_Account_Name,
                                                         account_key= storage_secret,
                                                         overwrite=True)

### Register a Datastore to link your Azure SQL Server to your AMLS Workspace
A Datastore is a pointer to an Azure Storage Account or Azure SQL Database that links to your AMLS Workspace.<br>
In the case of a SQL Server, you connect directly to a blob container.  You will need the following:<br>
- SQL Server Name
- SQL Database Name
- SQL User Name
- SQL User Password

In [ ]:
# Retrieve your Username and PasswordSecrets from Keyvault
user_secret = kv.get_secret(SQL_User_Secret)
sql_password_secret = kv.get_secret(SQL_Password_Secret)

In [ ]:
# Set up a Datastore using the normal parameters for Azure SQL Database
sql_datastore = Datastore.register_azure_sql_database(
       workspace=ws,
       datastore_name= SQL_Datastore_Name,
       server_name= SQL_Server_Name,
       database_name= SQL_Database_Name,
       username= user_secret,
       password= sql_password_secret)

### Create a Compute Cluster for Remote Training
Compute Clusters let your Train Machine Learning Models remotely and automatically scale up and down.<br>
Setting your minimum nodes to 0 will automatically spin down your cluster when it's not in use.<br>
Intelligently set your idle seconds before scaledown to control costs when you're not using your cluster.

In [ ]:
# Check to see if your Cluster already exists.  Spin it up if it does not.
if Compute_Cluster_Name in ws.compute_targets:
    compute_target = ws.compute_targets[Compute_Cluster_Name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + Compute_Cluster_Name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=Compute_Cluster_Type,
                                                                min_nodes=Compute_Cluster_Min_Node_Count,
                                                                max_nodes=Compute_Cluster_Max_Node_Count,
                                                                idle_seconds_before_scaledown = Idle_Seconds)
    # Create the cluster
    compute_target = ComputeTarget.create(ws, Compute_Cluster_Name, provisioning_config)
      
    # Wait for the cluster to complete, show the output log
    compute_target.wait_for_completion(show_output=True)

### Create an AKS Cluster for Real-Time Inferencing
Select a VM Size and number of VMs to run your AKS cluster.  Your cores must number at least 12, so if you select a VM with 4 cores, set agent count to 3.<br>
A single AKS Cluster can support many deployments depending on the size of the cluster.

Check out AKS configuration options here:<br> https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.aksprovisioningconfiguration?view=azure-ml-py

In [ ]:
# Set your configurations here.  Select Larger VM sizes and more agents for faster performance
# Agent Count x Number of Cores must be equal to or greater than 12
prov_config = AksCompute.provisioning_configuration(vm_size = AKS_Cluster_Type, agent_count = AKS_Node_Count,\
                                                    location = AKS_Location)

In [ ]:
# Check to see if your AKS Cluster already exists.  Spin it up if it does not.
if AKS_Cluster_Name in ws.compute_targets:
    aks_target = ws.compute_targets[AKS_Cluster_Name]
    if AKS_Cluster_Name and type(AKS_Cluster_Name) is AksCompute:
        print('found compute target. just use it. ' + AKS_Cluster_Name)
else:
    print('creating a new compute target...')
   # Create the cluster and attach it to your workspace
    aks_target = ComputeTarget.create(workspace = ws,
                                        name = AKS_Cluster_Name,
                                        provisioning_configuration = prov_config)
    # Wait for the create process to completeaks
    aks_target.wait_for_completion(show_output = True)